# Concept Activation Tensor (CAT)

> reproducing a CAT from the original paper : https://proceedings.mlr.press/v80/kim18d/kim18d.pdf

In [90]:
import os

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.optimizers import Adam, RMSprop

In [91]:
model = keras.models.load_model('./models/painter_model')

In [92]:
df = pd.read_json('./data/painters_ds.json')
df.head(1)

,category,input,output
http://dbpedia.org/resource/Alexander_Semionov,[http://dbpedia.org/resource/Russia],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [93]:
dfs = []
for filename in os.listdir(os.path.join('activations', 'painters_ds')):
    if filename.split('.')[-1] == 'pkl':
        dfs.append(pd.read_pickle(os.path.join('activations', 'painters_ds', filenameb)))

NameError: name 'filenameb' is not defined

In [ ]:
def get_activations(df, cat=None):
    activations_cols = [col for col in df if "neuron" in col]
    if cat is None:
        return df.loc[:, df.columns.isin(activations_cols)]
    else:
        return df[df.category.apply(lambda x: cat in x)].loc[:, df.columns.isin(activations_cols)]

In [ ]:
cat = "http://dbpedia.org/resource/Russia"

In [ ]:
udf = dfs[0]

In [ ]:
# X
X = get_activations(udf).to_numpy()
X

In [ ]:
def get_output(df, cat=None):
    if cat is None:
        return df.true
    else:
        return df[df.category.apply(lambda x: cat in x)].true

In [ ]:
# y
y = get_output(udf).to_numpy()
y

In [ ]:
len(X), len(y) # Check if has the same length

In [ ]:
X[0].shape

In [ ]:
# train_test_split
# already standardized but to check
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Re-creating model to predict classes from activations

In [94]:
# Must pass dynamic shapes
pa_model = Sequential() # pa stands for Predict from Activations
pa_model.add(keras.Input(shape=(64,)))
pa_model.add(Dense(64, activation='relu'))
pa_model.add(Dense(1, activation='softmax'))

pa_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


In [95]:
pa_model.compile(optimizer=Adam(), loss='binary_crossentropy', 
                 metrics=['accuracy'])
EPOCHS = 100
BATCH_SIZE = 64
history = pa_model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/100
83/83 [==============================] - 0s 767us/step - loss: 0.3899 - accuracy: 0.1646
Epoch 2/100
83/83 [==============================] - 0s 687us/step - loss: 0.3003 - accuracy: 0.1646
Epoch 3/100
83/83 [==============================] - 0s 768us/step - loss: 0.2790 - accuracy: 0.1646
Epoch 4/100
83/83 [==============================] - 0s 695us/step - loss: 0.2674 - accuracy: 0.1646
Epoch 5/100
83/83 [==============================] - 0s 738us/step - loss: 0.2624 - accuracy: 0.1646
Epoch 6/100
83/83 [==============================] - 0s 736us/step - loss: 0.2538 - accuracy: 0.1646
Epoch 7/100
83/83 [==============================] - 0s 731us/step - loss: 0.2503 - accuracy: 0.1646
Epoch 8/100
83/83 [==============================] - 0s 757us/step - loss: 0.2464 - accuracy: 0.1646
Epoch 9/100
83/83 [==============================] - 0s 716us/step - loss: 0.2451 - accuracy: 0.1646
Epoch 10/100
83/83 [==============================] - 0s 736us/step - loss: 0.2436 - accura

83/83 [==============================] - 0s 901us/step - loss: 0.2007 - accuracy: 0.1646
Epoch 82/100
83/83 [==============================] - 0s 708us/step - loss: 0.2024 - accuracy: 0.1646
Epoch 83/100
83/83 [==============================] - 0s 719us/step - loss: 0.2018 - accuracy: 0.1646
Epoch 84/100
83/83 [==============================] - 0s 727us/step - loss: 0.2033 - accuracy: 0.1646
Epoch 85/100
83/83 [==============================] - 0s 740us/step - loss: 0.2032 - accuracy: 0.1646
Epoch 86/100
83/83 [==============================] - 0s 723us/step - loss: 0.2032 - accuracy: 0.1646
Epoch 87/100
83/83 [==============================] - 0s 758us/step - loss: 0.2031 - accuracy: 0.1646
Epoch 88/100
83/83 [==============================] - 0s 709us/step - loss: 0.1973 - accuracy: 0.1646
Epoch 89/100
83/83 [==============================] - 0s 725us/step - loss: 0.1969 - accuracy: 0.1646
Epoch 90/100
83/83 [==============================] - 0s 761us/step - loss: 0.1998 - accuracy: 